In [4]:
from PIL import Image
import requests

import torch
from torch import nn
from transformers import AutoProcessor, CLIPVisionModel, CLIPVisionConfig, CLIPPreTrainedModel
from transformers.models.clip.modeling_clip import CLIPVisionModelOutput, CLIPVisionTransformer
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

[2024-01-24 15:18:49,948] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
from llava_phi import LlavaPhiForCausalLM

In [3]:
# model = LlavaPhiForCausalLM.from_pretrained("./llava-phi/checkpoints/llavaPhi-v0-3b-finetune/checkpoint-4000")

In [4]:
# processor = AutoProcessor.from_pretrained("./llava-phi/checkpoints/llavaPhi-v0-3b-finetune/checkpoint-4000")

In [5]:
# prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
# url = "https://www.ilankelman.org/stopsigns/australia.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

In [6]:
# # Generate
# generate_ids = model.generate(**inputs, max_length=30)
# processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [6]:
from llava_phi.model.builder import load_pretrained_model
from llava_phi.mm_utils import tokenizer_image_token, get_model_name_from_path
from llava_phi.utils import disable_torch_init
from llava_phi.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava_phi.conversation import conv_templates, SeparatorStyle

In [11]:
model_path = "checkpoints/llavaPhi-v0-3b-finetune/checkpoint-4000"
model_name = get_model_name_from_path(model_path)

In [12]:
model_name

'llavaPhi-v0-3b-finetune_checkpoint-4000'

In [13]:
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, None, model_name)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


load llaVA-Phi MLLM!!!


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'device_map': 'cuda'}


In [14]:
qs = "What's the content of the image?"
qs = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + qs

In [15]:
conv = conv_templates["default"].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

In [16]:
prompt

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <im_start><image><im_end>\nWhat's the content of the image? ASSISTANT:"

In [17]:
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].cuda()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2

In [25]:
input_ids.shape

torch.Size([1, 55])

In [24]:
from datasets import load_dataset
audio_ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio = audio_ds[0]["audio"]

whisper_w_proj = WhisperWithProjection(projection_dim=512)
audio_embed = whisper_w_proj(audio)["input_ids"]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
audio_embed.shape, audio_embed.device

(torch.Size([1, 33]), device(type='cpu'))

In [29]:
input_ids = torch.concat([input_ids, audio_embed.to("cuda:0")], dim=1)

In [30]:
input_ids.shape

torch.Size([1, 88])

In [31]:
with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=0.2,
            max_new_tokens=1024,
            eos_token_id=tokenizer.eos_token_id,  # End of sequence token
            pad_token_id=tokenizer.eos_token_id,  # Pad token
            use_cache=True,
        )

In [32]:
input_token_len = input_ids.shape[1]
n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
if n_diff_input_output > 0:
    print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
outputs = outputs.strip()
if outputs.endswith(stop_str):
    outputs = outputs[:-len(stop_str)]
outputs = outputs.strip()
print(outputs)

is a Japanese character from the story of Jesus, who is a Chinese monk who is also known for his teachings. The story is based on the story of the story of Jesus Christ, and it is a representation of the story of Jesus and the story of Jesus Christ.


In [20]:
# image

In [52]:
class AudioLanguageConnector:
    def __init__(self, projection_dim):
        model_name = "microsoft/phi-2"
        self.phi2_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.phi2_tokenizer.pad_token = self.phi2_tokenizer.eos_token
        self.phi2_tokenizer.max_length = projection_dim

    def __call__(self, text):
        text = f"<audio_start> {text} <audio_end>"
        tokens = self.phi2_tokenizer(text, return_tensors="pt", return_attention_mask=False)
        return tokens
        

class WhisperWithProjection:
    def __init__(self, projection_dim, device):
        self.device = device
        self.processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", device_map=device)
        self.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny", device_map=device)
        self.model.config.forced_decoder_ids = None
        self.audio_language_connector = AudioLanguageConnector(projection_dim)
        
    def __call__(self, audio):
        input_features = self.processor(audio["array"],
                                   sampling_rate=audio["sampling_rate"],
                                   return_tensors="pt").input_features
        # generate token ids
        predicted_ids = self.model.generate(input_features.to(self.device))
        # decode token ids to text        
        transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)

        audio_embeddings = self.audio_language_connector(transcription)
        return audio_embeddings.to(self.device)

In [53]:
class MultiModalPhi2:
    def __init__(self, model_path="checkpoints/llavaPhi-v0-3b-finetune/checkpoint-4000",
                temperature=0.2,
                max_new_tokens=1024,
                device="cuda"):
        self.temperature = temperature
        self.max_new_tokens = max_new_tokens
        self.device = device
        model_name = get_model_name_from_path(model_path)
        self.tokenizer, self.model, self.image_processor, self.context_len = load_pretrained_model(model_path, None, model_name, device_map=device)
        self.whisper_w_proj = WhisperWithProjection(projection_dim=512, device=device)
        
        
    def __call__(self, text, audio, image):
        qs = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + text
        conv = conv_templates["default"].copy()
        conv.append_message(conv.roles[0], qs)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        image_tensor = self.image_processor.preprocess(image, return_tensors='pt')['pixel_values'].cuda()
        
        input_ids = tokenizer_image_token(prompt, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()

        audio_embed = self.whisper_w_proj(audio)["input_ids"]
        
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2

        input_ids = torch.concat([input_ids, audio_embed], dim=1)

        with torch.inference_mode():
            output_ids = self.model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=self.temperature,
                max_new_tokens=self.max_new_tokens,
                eos_token_id=tokenizer.eos_token_id,  # End of sequence token
                pad_token_id=tokenizer.eos_token_id,  # Pad token
                use_cache=True,
            )

        input_token_len = input_ids.shape[1]
        n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
        if n_diff_input_output > 0:
            print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
        outputs = self.tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
        outputs = outputs.strip()
        if outputs.endswith(stop_str):
            outputs = outputs[:-len(stop_str)]
        outputs = outputs.strip()
        return outputs

In [54]:
multimodal_phi2 = MultiModalPhi2()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


load llaVA-Phi MLLM!!!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'device_map': 'cuda'}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [57]:
text = "tell me about the audio"

In [58]:
multimodal_phi2(text, audio, image)

'is a popular Japanese drama series featuring a man in a red and white costume, who is dressed as Santa Claus, is walking down the street. The scene takes place in a busy city environment, with people walking and standing on the sidewalk, likely enjoying the festive atmosphere and the festive atmosphere.'

In [64]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig, CLIPImageProcessor

In [65]:
from llava_phi.model import LlavaPhiConfig

In [66]:
device_map = "cuda:0"
load_8bit = False
load_4bit = False
kwargs = {"device_map": device_map}
if load_8bit:
    kwargs['load_in_8bit'] = True
elif load_4bit:
    kwargs['load_in_4bit'] = True
    kwargs['quantization_config'] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
config = LlavaPhiConfig.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = LlavaPhiForCausalLM.from_pretrained(
    model_path, 
    config=config, 
    use_safetensors=True, 
    **kwargs).to("cuda")
image_processor = CLIPImageProcessor.from_pretrained(model_path)
mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
mm_use_im_patch_token = getattr(model.config, "mm_use_im_patch_token", True)

# TODO: the tokenizer length of phi-2 is 50295, but the output class of lm_head is 51200
if mm_use_im_patch_token:
    tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
if mm_use_im_start_end:
    tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)
    
if hasattr(model.config, "max_sequence_length"):
        context_len = model.config.max_sequence_length
else:
    context_len = 2048

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [70]:
model.save_pretrained("Llava-Phi-Checkpoint")

KeyboardInterrupt: 

In [ ]:
model.push_to_hub("RaviNaik/Llava-Phi2")

In [73]:
tokenizer.push_to_hub("RaviNaik/Llava-Phi2")

CommitInfo(commit_url='https://huggingface.co/RaviNaik/Llava-Phi2/commit/fa8f7240058241243f6bdc3d6ab44bb691f76e39', commit_message='Upload tokenizer', commit_description='', oid='fa8f7240058241243f6bdc3d6ab44bb691f76e39', pr_url=None, pr_revision=None, pr_num=None)